In [ ]:
%matplotlib inline
from nn_mri import ImageFitting_set, Siren, get_mgrid, cases, calculate_contrast, save_dicom
import numpy as np
import torch
from PIL import Image
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from skimage.color import rgb2gray, gray2rgb

from skimage import data, img_as_float
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.metrics import peak_signal_noise_ratio
from skimage.util import random_noise
from skimage.transform import rescale, resize, downscale_local_mean
import os
import SimpleITK as sitk




total_steps = 3000
gland_start = 0
focus_size = 128
show_cancer = False
weighted = False
display = False
do_detection = False
sigma_est = 2
scale = 1
patch_kw = dict(patch_size=3,      # 5x5 patches
                patch_distance=3)
metrics = ['C', 'CNR']
filename = '../experiments/f.csv'
with open(filename, 'w') as f:
    f.write('seed,patient,direction,epoch,image,metric,performance\n')




In [ ]:
#!pip install scikit-learn
!pip install dipy

In [ ]:
from nn_mri import cases, calculate_contrast, save_dicom
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from skimage.restoration import denoise_nl_means
from dipy.denoise.nlmeans import nlmeans
from dipy.denoise.noise_estimate import estimate_sigma
import numpy as np
patch_kw = dict(patch_size=5, patch_distance=3)

accept = np.ones((3,4), dtype=bool)
for case in cases:
    
    sigma_est =  estimate_sigma(case.dwi, N=8)
    
    _slice = case.cancer_slice
    orig = case.dwi[:, :, _slice, :].mean(-1)

    new = orig.copy()
#     for i in range(case.dwi.shape[3]):
#         img = case.dwi[:, :, _slice, i]
#         print(case.pt_id, calculate_contrast(case, 1, img, 0))
    print('orig ',case.pt_id, calculate_contrast(case, 1, orig, 0))

    img = case.dwi[:, :, _slice, :]
    for i in range(2,127):
        for j in range(2,127):
            acq = img[i,j,:].reshape(-1,1)
            db = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='single').fit(acq)
            sample_means = [acq[db.labels_== x].mean() for x in set(db.labels_)]
            sample_lens = [len(acq[db.labels_== x]) for x in set(db.labels_)]
            for k in range(2):
                if sample_means[k] > sample_means[1-k] and sample_lens[k] > (1/3)*case.dwi.shape[3]:
                    case.accept[i, j, _slice, :] = (db.labels_== k)
                    #case.change_accept(i, j, _slice, (db.labels_== k))
            #print(img.shape)
            #for i in range(orig)
    print('clus', case.pt_id, calculate_contrast(case, 1, new, 0))
    plt.figure()
    plt.imshow(orig, cmap='gray')
    plt.figure()
    plt.imshow(new, cmap='gray')


In [ ]:
den = nlmeans(case.dwi, sigma=sigma_est, patch_radius=1, block_radius=2, rician=True)
filtered = den[:, :, _slice, :].mean(-1)
print('clus', case.pt_id, calculate_contrast(case, 1, new, 0))

plt.figure()
plt.imshow(orig[40:90, 40:90], cmap='gray')
plt.figure()
plt.imshow(new[40:90, 40:90], cmap='gray')
plt.figure()
plt.imshow(filtered[40:90, 40:90], cmap='gray')